# FTX reader

In [1]:
import matplotlib
from datetime import *
%matplotlib inline
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import plotly.express as px
import nest_asyncio
nest_asyncio.apply()
import os,sys
sys.path.append('../')
from utils.io_utils import *
from histfeed.ftx_history import main

end_time=datetime.now(timezone.utc)-timedelta(days=1)#.replace(minute=0,second=0,microsecond=0)
start_time=end_time-timedelta(days=7)
halflife=timedelta(days=1)
modes=[('h','high'),('c','close'),('l','low'),('v','size')]

In [20]:
weights = pd.read_excel(os.path.join(os.sep, 'home','david',"config", "pfoptimizer",'current_weights.xlsx')).sort_values(by='optimalWeight',key=lambda f: np.abs(f),ascending=False)
coins = ['ETH','AAVE']#list(weights.set_index(['name']).drop(['USD','total']).head(5).index.map(lambda f: f.replace('-PERP','')))
weights

,name,spotBenchmark,ExpectedBenchmark,FundingBenchmark,currentWeight,optimalWeight,ExpectedCarry,RealizedCarry,excessIM,excessMM,transactionCost,time
2,USD,0.026772,0.026772,0.000073,3324.738638,2976.505318,0.000000,0.000000,2930.666630,2564.559007,0,2022-06-20 09:00:00
1,RSR-PERP,-0.005266,-0.004333,-0.006144,0.000000,-2059.729343,2181.824807,0.000000,-2779.948839,-2564.557370,0,2022-06-20 09:00:00
3,total,0.001189,-0.129723,-0.000599,916.773762,916.773762,2181.824821,-6197.974852,150.715125,-0.000823,0,2022-06-20 09:00:00
0,BTC-PERP,0.000098,-0.000110,0.000048,-16.387200,-0.001858,-0.000050,-0.438716,-0.002189,-0.002018,0,2022-06-20 09:00:00


In [12]:
futures_df = main([None,'get','ftx','wide'])

INFO:histfeed.ftx_history:Logger set
INFO:histfeed.ftx_history:Cannot find the nb_of_days param. Defaulting to nb_of_days=100
INFO:histfeed.ftx_history:histfeed running with params ['get', 'ftx', 'wide']
INFO:histfeed.ftx_history:Getting history...


### rates markets

In [21]:
futures_df.columns

Index(['1INCH-PERP_rate_funding', 'AAVE-PERP_rate_funding',
       'AVAX-PERP_rate_funding', 'AXS-PERP_rate_funding',
       'BCH-PERP_rate_funding', 'BNB-PERP_rate_funding',
       'BTC-PERP_rate_funding', 'DOGE-PERP_rate_funding',
       'DOT-PERP_rate_funding', 'ETH-PERP_rate_funding',
       ...
       'UNI_rate_borrow', 'UNI_rate_size', 'USDT_rate_borrow',
       'USDT_rate_size', 'XRP_rate_borrow', 'XRP_rate_size', 'YFI_rate_borrow',
       'YFI_rate_size', 'USD_rate_borrow', 'USD_rate_size'],
      dtype='object', length=646)

In [23]:
selected = [df for df in futures_df.columns if any(df.split('-')[0].split('_')[0]==coin for coin in (coins+['USD']))]
prices=futures_df[[c for c in selected if any(string in c for string in ['price_c'])]]

short_rates=futures_df[[c for c in selected if any(string in c for string in ['PERP_rate_c','rate_borrow'])]]
short_rates=short_rates.ewm(halflife=halflife,times=short_rates.index).mean()

borrow_size=futures_df[[c for c in selected if any(string in c for string in ['_rate_size','price_volume'])]]

for c in coins:
    long_perp_carry=short_rates[c+'-PERP_rate_c']-short_rates['USD_rate_borrow']
    short_perp_carry=-short_rates[c+'-PERP_rate_c']-short_rates[c+'_rate_borrow']
    short_rates[c+'_best_perp_carry']= pd.concat([long_perp_carry, short_perp_carry]).max(level=0)
   # long_rates[c+'_long_future_carry']=long_rates[c+'-1231_rate_c']-short_rates['USD_rate_borrow']
   # long_rates[c+'_short_future_carry']=-long_rates[c+'-1231_rate_c']-short_rates[c+'_rate_borrow']

short_rates
pd.concat([short_rates],axis=0).iplot()#))

/tmp/ipykernel_45466/1800538636.py:12: FutureWarning:

Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().

/tmp/ipykernel_45466/1800538636.py:12: FutureWarning:

Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().



In [24]:
pd.concat([short_rates.filter(like='borrow')],axis=0).iplot()#))

In [25]:
pd.concat([borrow_size],axis=0).iplot()#))

### carry directionality

In [ ]:
plotthat = futures_df[['BNB-PERP/rate/h',
       'BNB-PERP/rate/l', 'BNB-PERP/rate/c']]
plotthat.columns=['h','l','c']

In [ ]:
import plotly.graph_objects as go
import numpy as np
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=plotthat.index,#+x_rev,
    y=0.5*(plotthat['l'].values+plotthat['h'].values),
    fill='toself',
    fillcolor='rgba(0,100,80,0.2)',
    line_color='rgba(255,255,255,0)',
    showlegend=False,
    name='BNB',
))
fig.add_trace(go.Scatter(
    x=plotthat.index, y=plotthat['c'].values,
    line_color='rgb(0,100,80)',
    name='Fair',
))

fig.update_traces(mode='lines')
fig.show()

In [ ]:
pd.concat([prices,short_rates.filter(like='carry')],axis=0).iplot(secondary_y=list(prices.columns))

# stopout: show min/max of premiums

In [ ]:
end_time=datetime.now(timezone.utc)#.replace(minute=0,second=0,microsecond=0)
start_time=end_time-timedelta(hours=1)
granular_df=ftx_history(dirname='',start=start_time,end=end_time,timeframe='15s',coin_list=coins)

In [ ]:
def compute_premium(df: pd.DataFrame,mode=1) -> pd.Series:
    return (df.filter(like='/mark/'+ modes[mode][0]).iloc[:,0]/df.filter(like='/indexes/'+modes[mode][1]).iloc[:,0]-1)*10000
def compute_premium_vs_spot(df: pd.DataFrame,mode=1) -> pd.Series:
    return (df.filter(like='/mark/'+ modes[mode][0]).iloc[:,0]/df.filter(like='/price/'+modes[mode][0]).iloc[:,0]-1)*10000
def compile_list(premium_list: list) -> pd.DataFrame:
    premium=premium_list[0]
    for df in premium_list[1:]:
        premium=premium.join(df,how='outer')
    return premium[~premium.index.duplicated()]

In [ ]:
granular_df['premium']=compute_premium_vs_spot(granular_df)
prices=granular_df[[c+'/price/c' for c in coins]]
granular_df[['premium']].iplot()

In [ ]:
premium=compute_premium(granular_df,mode=0)
ewMax=premium.resample('1h').max()
ewMax=ewMax[~ewMax.index.duplicated()]
pd.concat([prices,ewMax],axis=0).iplot(secondary_y=list(prices.columns))

In [ ]:
premium=compute_premium(granular_df,mode=2)
ewMin=premium.resample('1h').min()
ewMin=ewMin[~ewMin.index.duplicated()]
pd.concat([prices,ewMin],axis=0).iplot(secondary_y=list(prices.columns))